In [ ]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import numpy as np

import os

from ms_stim_analysis.Analysis.utils import filter_opto_data, smooth
from ms_stim_analysis.AnalysisTables.ms_opto_stim_protocol import OptoStimProtocol

from ms_stim_analysisAnalysisTables.decoding_tables import (
    ContinuousRippleTraversal,
    ContinuousRippleTraversalSelection,
    ContinuousRippleTraversalParams,
)
from spyglass.decoding.v1.clusterless import ClusterlessDecodingV1
from spyglass.ripple.v1 import RippleTimesV1
from spyglass.position.position_merge import PositionOutput



# Populate

In [ ]:
params = {
    "ripple_traversal_params_name": "default acausal",
    "continuous_distance_threshold": 3,
    "acausal": True,
}
ContinuousRippleTraversalParams().insert1(params, skip_duplicates=True)
ContinuousRippleTraversalParams()

In [ ]:
from ms_stim_analysis.Analysis.utils import filter_opto_data

artifact_detection = "ms_lfp_artifact_times"
dataset_key = {"track_type": "early_wtrack"}
dataset = filter_opto_data(dataset_key)
dataset = dataset - "nwb_file_name LIKE 'banner%'"
pos_key = (PositionOutput.DLCPosV1()).proj(pos_merge_id="merge_id")
for key in dataset.fetch("KEY"):
    clusterless_query = ClusterlessDecodingV1() & {
        "nwb_file_name": key["nwb_file_name"],
        "position_group_name": "DLC " + key["interval_list_name"],
        "encoding_interval": key["interval_list_name"],
    }
    if not clusterless_query:
        continue
        # raise ValueError("No clusterless decoding found")
    clusterless_key = clusterless_query.fetch1("KEY")
    clusterless_key
    ripple_key = {
        "nwb_file_name": key["nwb_file_name"],
    }
    ripple_query = (
        RippleTimesV1
        & ripple_key
        # & f"target_interval_list_name LIKE '%{key['interval_list_name']}%'"
        & f"target_interval_list_name LIKE '%{key['interval_list_name']}_{artifact_detection}%'"
        & pos_key
    )
    if not ripple_query:
        continue
        raise ValueError("No ripples found")
    ripple_key = ripple_query.fetch1("KEY")

    insert_key = {
        **ripple_key,
        **clusterless_key,
        "ripple_traversal_params_name": "default acausal",
    }
    ContinuousRippleTraversalSelection().insert1(insert_key, skip_duplicates=True)

    ContinuousRippleTraversalSelection()
ContinuousRippleTraversal().populate(processes=32)